In [1]:
import requests
import time
from bs4 import BeautifulSoup as bs
import datetime
import pandas as pd
import time

In [2]:
class Connect:
    def __init__(self):
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        
    def req(self, url, timeout=20):
        try:
            res = requests.get(url, timeout=timeout, headers=self.headers)
            return res
        except:
            print('Request Error,', datetime.datetime.now())
            return -1

In [3]:
class Preprocessor:
    def __init__(self, res):
        self.res = res
        
    def get_new_reports(self):
        soup = bs(self.res.text, features='html.parser')
        um_tdinsm = soup.find_all('dl', attrs={'class':'um_tdinsm'})
        gbox = soup.find_all('span', attrs={'class':'gpbox'})
        rs = soup.find_all('td', attrs={'class':'r'})
        clfs = soup.find_all('td', attrs={'class':'clf'})

        invest_opinion = [x.text.strip() for x in gbox[::3]]
        price_goal = [x.text.strip() for x in gbox[1::3]]
        dates = [x.text.strip() for x in clfs]
        reporter = []
        for i in gbox[2::3]:
            reporter.append(''.join([str(x) for x in i.contents[1:]]).replace('<br/>', ' ').strip())
        price_before = [x.text.strip() for x in rs[1::2]]
        titles = []
        contents = []
        for ele in um_tdinsm:
            title = ele.find('dt').text
            content = '\n- '.join([x.text.strip() for x in ele.findAll('dd')])
            titles.append(title)
            contents.append(content)
        df = pd.DataFrame({
            'invest_opinion': invest_opinion,
            'price_goal': price_goal,
            'reporter': reporter,
            'titles': titles,
            'contents': contents,
            'dates': dates
        })
        return df

In [4]:
start_date_str = '20220101'
end_date_str = '20221231'
start_date = datetime.datetime.strptime(start_date_str, "%Y%m%d")
end_date = datetime.datetime.strptime(end_date_str, "%Y%m%d")

In [5]:
df = pd.DataFrame()
while True:
    fr_dt = start_date.strftime('%Y%m%d')
    start_date += datetime.timedelta(days=10)
    to_dt = start_date.strftime('%Y%m%d')
    if to_dt > end_date_str:
        to_dt = end_date_str
    print(fr_dt, to_dt)
    url = f'https://comp.fnguide.com/svo2/asp/SVD_Report_Summary_Data.asp?fr_dt={fr_dt}&to_dt={to_dt}&stext=&check=all&_={int(time.time()*1000)}'
    connector = Connect()
    res = connector.req(url)
    ps = Preprocessor(res)
    df_part = ps.get_new_reports()
    df = pd.concat([df, df_part])
    if to_dt == end_date_str:
        break
    time.sleep(5)

20220101 20220111
20220111 20220121
20220121 20220131
20220131 20220210
20220210 20220220
20220220 20220302
20220302 20220312
20220312 20220322
20220322 20220401
20220401 20220411
20220411 20220421
20220421 20220501
20220501 20220511
20220511 20220521
20220521 20220531
20220531 20220610
20220610 20220620
20220620 20220630
20220630 20220710
20220710 20220720
20220720 20220730
20220730 20220809
20220809 20220819
20220819 20220829
20220829 20220908
20220908 20220918
20220918 20220928
20220928 20221008
20221008 20221018
20221018 20221028
20221028 20221107
20221107 20221117
20221117 20221127
20221127 20221207
20221207 20221217
20221217 20221227
20221227 20230106


In [8]:
df.to_csv('data/report_220101_221231.csv', index=False)

### workspace

In [24]:
soup = bs(res.text, features='html.parser')
um_tdinsm = soup.find_all('dl', attrs={'class':'um_tdinsm'})
gbox = soup.find_all('span', attrs={'class':'gpbox'})
rs = soup.find_all('td', attrs={'class':'r'})
clfs = soup.find_all('td', attrs={'class':'clf'})

In [47]:
len()

86

In [45]:
soup.find_all('td', attrs={'class':'clf'})[0].text

'2023/06/01'

In [37]:
gbox[3]

<span class="gpbox"> <img alt="유지" class="gp_img hold" src="../img/shbg.png"/>BUY</span>

In [25]:
invest_opinion = [x.text.strip() for x in gbox[::3]]
price_goal = [x.text.strip() for x in gbox[1::3]]
reporter = []
for i in gbox[2::3]:
    reporter.append(''.join([str(x) for x in i.contents[1:]]).replace('<br/>', ' ').strip())
price_before = [x.text.strip() for x in rs[1::2]]
titles = []
contents = []
for ele in um_tdinsm:
    title = ele.find('dt').text
    content = '\n- '.join([x.text.strip() for x in ele.findAll('dd')])
    titles.append(title)
    contents.append(content)

In [26]:
df = pd.DataFrame({
            'invest_opinion': invest_opinion,
            'price_goal': price_goal,
            'reporter': reporter,
            'titles': titles,
            'contents': contents
        })

In [ ]:
result = {}
for i in range(len(titles)):
    if title not in visited:
        text = f"{titles[i]}\n\n- {contents[i]}\n\n투자의견: {invest_opinion[i]}\n목표주가: {price_goal[i]}\n전일종가: {price_before[i]}\n제공처/작성자: {reporter[i]}"
        result[titles[i]] = text
return result

In [16]:
ps = Preprocessor(res)

In [17]:
url

'https://comp.fnguide.com/svo2/asp/SVD_Report_Summary_Data.asp?fr_dt=20230101&to_dt=20230531&stext=&check=all&_=1686329091734'

In [18]:
ps.get_new_reports({})

{}